In [1]:
import tensorflow as tf
from tensorflow import data as tfdata

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = tf.random.normal((num_examples, num_inputs), stddev=1)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b
labels += tf.random.normal(labels.shape,stddev=0.01)

In [2]:
batch_size = 10
# 将训练数据的特征和标签组合, 分成一组一组的
dataset = tfdata.Dataset.from_tensor_slices((features, labels))
# 随机打乱数据索引，每次选buffer_size个数据 https://blog.csdn.net/dpengwang/article/details/103799449
dataset = dataset.shuffle(buffer_size=num_examples) 
# 分成若干组数据，每组数据大小为batch_size
dataset = dataset.batch(batch_size)
# 创建迭代器
#data_iter = iter(dataset)

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import initializers as init

# Sequential实例可以看作是一个串联各个层的容器。
model = keras.Sequential()
# 输出维度为1 指定权重参数每个元素将在初始化时随机采样于均值为0、标准差为0.01的正态分布
model.add(layers.Dense(1, kernel_initializer=init.RandomNormal(stddev=0.01)))


In [4]:
# 定义损失函数
from tensorflow import losses
loss = losses.MeanSquaredError()

# 定义优化算法
from tensorflow.keras import optimizers
trainer = optimizers.SGD(learning_rate=0.03)

num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for (batch, (X, y)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            l = loss(model(X, training=True), y)

        # 获得动态图中各变量梯度
        grads = tape.gradient(l, model.trainable_variables)
        # apply_gradients 跟新权重
        # zip将对象中对应的元素打包成一个个元组 https://www.runoob.com/python3/python3-func-zip.html
        trainer.apply_gradients(zip(grads, model.trainable_variables))

    l = loss(model(features), labels)
    print('epoch %d, loss: %f' % (epoch, l))


epoch 1, loss: 0.000231
epoch 2, loss: 0.000094
epoch 3, loss: 0.000094


In [6]:
true_w, model.get_weights()[0]

([2, -3.4],
 array([[ 1.9999132],
        [-3.3987668]], dtype=float32))

In [7]:
true_b, model.get_weights()[1]

(4.2, array([4.1996055], dtype=float32))

In [42]:
'''使用 Keras'''

# 生成数据
import tensorflow as tf
from tensorflow import data as tfdata
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = tf.random.normal((num_examples, num_inputs), stddev=1)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b
labels += tf.random.normal(labels.shape,stddev=0.01)

In [43]:
# 定义模型和初始化参数
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import initializers as init

model = keras.Sequential()
model.add(layers.Dense(1, kernel_initializer=init.RandomNormal(stddev=0.01), input_dim=2))

In [44]:
# 训练模型
from tensorflow.keras import losses
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping

# 定义优化器
sgd = SGD(lr=0.03)
model.compile(optimizers=sgd, loss=losses.mean_squared_error)

# 当被监测的数量不再提升，则停止训练 https://keras.io/zh/callbacks/
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, 
    patience=5, verbose=1, mode='auto', restore_best_weights=True)

model.fit(features, labels, epochs=100, batch_size=10, callbacks=[monitor])

Train on 1000 samples
Epoch 1/100
1000/1000 [==============================] - 1s 503us/sample - loss: 33.3036
Epoch 2/100
1000/1000 [==============================] - 0s 140us/sample - loss: 31.6566
Epoch 3/100
1000/1000 [==============================] - 0s 147us/sample - loss: 30.0375
Epoch 4/100
1000/1000 [==============================] - 0s 146us/sample - loss: 28.4376
Epoch 5/100
1000/1000 [==============================] - 0s 146us/sample - loss: 26.8950
Epoch 6/100
1000/1000 [==============================] - 0s 144us/sample - loss: 25.4125
Epoch 7/100
1000/1000 [==============================] - 0s 133us/sample - loss: 23.9316
Epoch 8/100
1000/1000 [==============================] - 0s 141us/sample - loss: 22.5311
Epoch 9/100
1000/1000 [==============================] - 0s 142us/sample - loss: 21.1742
Epoch 10/100
1000/1000 [==============================] - 0s 136us/sample - loss: 19.8939
Epoch 11/100
1000/1000 [==============================] - 0s 138us/sample - loss: 18.60

In [45]:
true_w, model.get_weights()[0]

([2, -3.4],
 array([[ 1.9998162],
        [-3.4016132]], dtype=float32))